### SongShu --Extraction of faimly relationships from bookmarks

The purpose of this script is to create a network of family relationships

In [1]:
import json
from SongShu969 import SongShu
import re
import pandas as pd

import types

In [2]:
Surname = None
from Surname import Surname

In [3]:
songshu = SongShu("2019-05-18", "RGT")
songshu.load_htmls("./Songshu969/")
songshu.extract_all()

INFO:root:Stop at loading ./Songshu969/SongShu_0969.html.
INFO:root:Total length of the data is 969.
ERROR:root:[Error] SongShu_rare_char.json does not exist

            try to run these lines: 
            	>> self.extract_rare_chars()
            	>> self.write_rare_chars()

INFO:root:Remove 標註, page number, and page dividers from the tree structure.
INFO:root:Remove the new lines added by the page dividers, connect the paragraphs before and after the new lines.


In [4]:
import re
from Surname import Surname
s = Surname()

In [5]:
# This removes the unnecessary info at the end of the bookmark
regex_clean_bookmark1 = re.compile(r"\(P\.\d+\)\.\.\[底本.+?\]$")  # (P.nnnn)..[底本：宋元明三朝遞修本]
Bookmarks = []
for bm in songshu.paths:
    bm = regex_clean_bookmark1.sub('', bm) # delete (P.nnnn)..[底本：宋元明三朝遞修本]
    Bookmarks.append(bm)

In [91]:
prots = re.compile(r"""列傳第\w+／  # the part that precedes the main protagonist
                            (\w+)       # main protagonist
                            (?:         # start of other character(s) subordinate to the main protagonist, optional
                             (?:
                              (?:[／　])? # separator of main protagonist and other persons; 2nd char in [] is \u3000
                              (.+?$)      # other person(s), as a string
                             )+
                            )?          # end of other characters""",
                        re.VERBOSE)

family = r"[長曾玄從族]?[兄弟伯叔子孫祖]"

def getProtagonists(bookmark):
    '''
    '''
    PROTS = []
    main_prot = None
    matches = prots.findall(bookmark)
    if len(matches) > 0:
        main_prot = matches[0][0]
        other_prots = matches[0][1]
    PROTS.append(main_prot)
    #print(f"other_prots={other_prots}")
    # get other characters
    if len(other_prots) > 0: # if other_prots is not an empty string
        characters = re.split(r'[／　]', other_prots)
        if len(characters) > 0:
            for c in characters:
                #print(f"c={c}")
                # the following returns a 3-tuple, (person, relationship, person)
                if re.match(fr"^{family}", c): # string starts with a family term
                    persons = [''] + list(re.findall(fr"^({family})(\w+)$", c)[0])
                else: # string does NOT start with a family term 
                      # then there are 2 possibilities (1) unrelated character, (2) family member
                    if not re.search(fr"{family}", c) : # test if this is an unrelated character; no familty term found
                        persons = c
                        #print(f"No family term found: {c}")
                    else:
                        #print(f"Family term found: {c}")
                        persons = list(re.findall(fr"^(\w+?)({family})(\w+)$", c)[0])
                #print(f"persons = {persons}")
                PROTS.append(persons)
        

    return PROTS
    #return [main_prot, characters]

In [92]:
Bionet = [None] * len(Bookmarks)
len(Bionet)

969

In [93]:
start_idx = 664 # 664
# example: people == ['張茂度', ['陸仲元'], ['仲元', '弟', '子真']]
for jj, b in enumerate(Bookmarks[start_idx:start_idx+80]):
    
    j = start_idx + jj
    Bionet[j] = {}

    print(j)
    print(b)
    people = getProtagonists(b)
    print(f"BEFORE: {people}")
    
    people2 = list(people)  # create a new copy
    # get persons who are standalone in the people list; e.g. 674 張茂度／陸仲元
    standalones = [s.split_name(person) for person in people if type(person) is not list]
    print(f"STANDALONES: {standalones}")
    for (sur0, giv0) in standalones:
        Bionet[j][f"{sur0}{giv0}"] = 1
    
    
    # now visit each remaining persons in the people list 
    # Pass (1): fix surname of A in ['A', 'B', 'C'] for non-empty A
    for i, p in enumerate(people): # skip the first n standalones
        if type(p) is not list:
            continue
        if len(p) > 1: # a list with 3 elements
            for (sur, giv) in standalones:
                if p[0] == giv:  # match given names
                    people2[i][0] = sur + people2[i][0]  # add surname to A
                    if people2[i][0] in Bionet[j]:
                        Bionet[j][people2[i][0]] += 1
                    else:
                        Bionet[j][people2[i][0]] = 1
                    people2[i][2] = sur + people2[i][2]  # add surname to C
                    if people2[i][2] in Bionet[j]:
                        Bionet[j][people2[i][2]] += 1
                    else:
                        Bionet[j][people2[i][2]] = 1
                    break # no need to check others

    #print(f"AFTER:  {people2}")
    #print("-"*30)
    #continue

    # Pass (2): for ['', 'B', 'C'], fill in '' and add surname to C
    for i, p in enumerate(people): # skip the first n standalones
        if type(p) is not list:
            continue
        if len(p) > 1: # a list with 3 elements
            if p[0] == '': # e.g. ['褚叔度', ['', '長兄', '秀之'], ['秀之', '弟', '淡之']]
                # check previous entry's surname
                previous_entry = people2[i-1]
                if type(previous_entry) is not list: # '褚叔度'
                    (sur2, giv2) = s.split_name(previous_entry)
                    people2[i][0] = previous_entry  # set A = previous
                    people2[i][2] = sur2 + people2[i][2]  # add surname to C
                    Bionet[j][people2[i][0]] = 1
                    Bionet[j][people2[i][2]] = 1
                else: # it's a list
                    try:
                        (sur2, giv2) = s.split_name(previous_entry[0])
                    except:
                        pass
                    if sur2 == None:
                        people2[i][0] = previous_entry[0]  # set A = previous
                    else:
                        people2[i][0] = previous_entry[0]  # set A = previous

    for i, p in enumerate(people): # skip the first n standalones
        if type(p) is not list:
            continue
        if len(p) > 1: # a list with 3 elements
            for person in Bionet[j].keys():
                #print(f"person={person}")
                (sur3, giv3) = s.split_name(person)
                if p[0] == giv3:
                    people2[i][0] = sur3 + giv3
                    people2[i][2] = sur3 + people2[i][2]
                #break

    print(f"AFTER:  {people2}")
    print("-"*30)

    #for i, p in enumerate(people[len(standalones):]): # skip the first n standalones
    #    if len(p) > 1: # a list with 3 elements
    #        if re.search(fr"{family}", p[1]): # family, same surname
    #            people2[i][2] = surname + people2[2]


664
史／正史／宋書／列傳　凡六十卷／卷五十二　列傳第十二／庾悅
BEFORE: ['庾悅']
STANDALONES: [('庾', '悅')]
AFTER:  ['庾悅']
------------------------------
665
史／正史／宋書／列傳　凡六十卷／卷五十二　列傳第十二／王誕
BEFORE: ['王誕']
STANDALONES: [('王', '誕')]
AFTER:  ['王誕']
------------------------------
666
史／正史／宋書／列傳　凡六十卷／卷五十二　列傳第十二／謝景仁
BEFORE: ['謝景仁']
STANDALONES: [('謝', '景仁')]
AFTER:  ['謝景仁']
------------------------------
667
史／正史／宋書／列傳　凡六十卷／卷五十二　列傳第十二／謝景仁／弟純
BEFORE: ['謝景仁', ['', '弟', '純']]
STANDALONES: [('謝', '景仁')]
AFTER:  ['謝景仁', ['謝景仁', '弟', '謝純']]
------------------------------
668
史／正史／宋書／列傳　凡六十卷／卷五十二　列傳第十二／謝景仁／弟述
BEFORE: ['謝景仁', ['', '弟', '述']]
STANDALONES: [('謝', '景仁')]
AFTER:  ['謝景仁', ['謝景仁', '弟', '謝述']]
------------------------------
669
史／正史／宋書／列傳　凡六十卷／卷五十二　列傳第十二／袁湛
BEFORE: ['袁湛']
STANDALONES: [('袁', '湛')]
AFTER:  ['袁湛']
------------------------------
670
史／正史／宋書／列傳　凡六十卷／卷五十二　列傳第十二／袁湛／弟豹
BEFORE: ['袁湛', ['', '弟', '豹']]
STANDALONES: [('袁', '湛')]
AFTER:  ['袁湛', ['袁湛', '弟', '袁豹']]
------------------------------
671
史／正史／宋書／列傳　凡六十卷／卷五十二　

TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [94]:
Bionet[start_idx:712]

[{'庾悅': 1},
 {'王誕': 1},
 {'謝景仁': 1},
 {'謝景仁': 1, '謝純': 1},
 {'謝景仁': 1, '謝述': 1},
 {'袁湛': 1},
 {'袁湛': 1, '袁豹': 1},
 {'褚叔度': 1, '褚秀之': 1},
 {'褚叔度': 1, '褚秀之': 1},
 {'張茂度': 1},
 {'張茂度': 1, '陸仲元': 2, '陸子真': 1},
 {'張茂度': 1, '張永': 1},
 {'庾登之': 1},
 {'庾登之': 1, '庾炳之': 1},
 {'謝方明': 1},
 {'謝方明': 1, '謝惠連': 1},
 {'江夷': 1},
 {'孔季恭': 1},
 {'孔季恭': 1, 'NoneNone': 1, '孔山士': 1},
 {'羊玄保': 1},
 {'羊玄保': 1, '褚胤': 1},
 {'羊玄保': 1, '羊子希': 1},
 {'沈曇慶': 1},
 {'臧燾': 1},
 {'臧燾': 1, '傅僧祐': 1},
 {'徐廣': 1},
 {'傅隆': 1},
 {'謝瞻': 1},
 {'謝瞻': 1, '謝㬭': 1},
 {'孔琳之': 1},
 {'孔琳之': 1, '孔道存': 1},
 {'蔡廓': 1},
 {'蔡廓': 1, '蔡興宗': 1},
 {'王惠': 1},
 {'謝弘微': 1},
 {'王球': 1},
 {'殷淳': 1},
 {'殷淳': 1, '殷孚': 1},
 {'殷淳': 1, 'NoneNone': 1, '殷沖': 1},
 {'張暢': 1},
 {'張暢': 1, '張淹': 1},
 {'張暢': 1, '張悅': 1},
 {'何偃': 1},
 {'江智淵': 1},
 {'范泰': 1},
 {'王准之': 1},
 {'王韶之': 1},
 {'NoneNone': 1}]

In [83]:
people2

['荀伯子', ['荀伯子', '子', '赤松']]

In [29]:
Bionet[start_idx:start_idx+20]

[{'褚叔度': 1, '褚秀之': 1},
 {'褚叔度': 1, '褚秀之': 1},
 {'張茂度': 1},
 {'張茂度': 1, '陸仲元': 2, '陸子真': 1},
 {'張茂度': 1, '張永': 1},
 {'庾登之': 1},
 {'庾登之': 1, '庾炳之': 1},
 {'謝方明': 1},
 {'謝方明': 1, '謝惠連': 1},
 {'江夷': 1},
 {'孔季恭': 1},
 {'孔季恭': 1, 'NoneNone': 1, '孔山士': 1},
 {'羊玄保': 1},
 {'羊玄保': 1, '褚胤': 1},
 {'羊玄保': 1, '羊子希': 1},
 {'沈曇慶': 1},
 {'臧燾': 1},
 {'臧燾': 1, '傅僧祐': 1},
 {'徐廣': 1},
 {'傅隆': 1}]

In [95]:
for bio in songshu.flat_bodies[start_idx:start_idx+1]:
    #print(bio.span.text)
    pass
songshu.flat_bodies[start_idx+5].find_all('h3')

[]

In [178]:
start_idx

664

In [99]:
Bionet[start_idx+5]

{'袁湛': 1}

In [205]:
fin = 'bio300to510AD_initial_text.txt'
with open(fin, "r", encoding='utf-8', newline='\n') as fi:
    bio = fi.read()

bio = bio.split('-'*20)
bio = [b.strip() for b in bio]

persons = []

for b in bio:
    #print(f'b={b}')
    p = b.split('\n')
    if (len(p) > 1):
        persons.append(p[1])
len(persons)

1088

In [174]:
Bionet[start_idx:start_idx+10]

[{'庾悅': 1},
 {'王誕': 1},
 {'謝景仁': 1},
 {'謝景仁': 1, '謝純': 1},
 {'謝景仁': 1, '謝述': 1},
 {'袁湛': 1},
 {'袁湛': 1, '袁豹': 1},
 {'褚叔度': 1, '褚秀之': 1},
 {'褚叔度': 1, '褚秀之': 1},
 {'張茂度': 1}]

In [208]:
start_idx = 664 # 669 # 664 

for j in range(0,10):
    print(start_idx+j, Bionet[start_idx+j])
    cnt = 0
    for paragraph in songshu.flat_bodies[start_idx+j].span.find_all('div'):
        print(paragraph.text)
        print('-'*30)
        line = paragraph.text
        for prot in Bionet[start_idx+j].keys(): # "internal" list
            (sur, giv) = s.split_name(prot)
            line = re.sub(fr"({prot})", f"<person name='{prot}'>{prot}</person>", line)
            line = re.sub(fr"({giv})(?!['<])", f"<person name='{prot}'>{giv}</person>", line)
            #line = re.sub(f"{family}", "\1", line)
        for prot in persons: # "external" list
            line = re.sub(fr"({prot})(?!['<])", f"<person name='{prot}'>{prot}</person>", line)
        print(line)
        print('='*30)


664 {'庾悅': 1}
庾悅字仲豫，潁川𨻳陵人也。曾祖亮，晉太尉。祖羲，吳國內史。父准，西中郎將、豫州刺史。
------------------------------
<person name='庾悅'>庾悅</person>字仲豫，潁川𨻳陵人也。曾祖亮，晉太尉。祖羲，吳國內史。父准，西中郎將、豫州刺史。
悅少為衞將軍琅邪王行參軍，司馬，徙主簿，轉右長史。桓玄輔政，領豫州，以悅為別駕從事史。遷驍騎將軍。玄篡位，徙中書侍郎。高祖定京邑，武陵王遵承制，以悅為寧遠將軍、安遠護軍、武陵內史。以病去職。
------------------------------
<person name='庾悅'>悅</person>少為衞將軍琅邪王行參軍，司馬，徙主簿，轉右長史。<person name='桓玄'>桓玄</person>輔政，領豫州，以<person name='庾悅'>悅</person>為別駕從事史。遷驍騎將軍。玄篡位，徙中書侍郎。高祖定京邑，武陵王遵承制，以<person name='庾悅'>悅</person>為寧遠將軍、安遠護軍、武陵內史。以病去職。
鎮軍府版諮議參軍，轉車騎從事中郎。劉毅請為撫軍司馬，不就。遷車騎中軍司馬。從征廣固，竭其誠力。盧循逼京都，以為督江州豫州之西陽新蔡汝南潁川司州之恒農揚州之松滋六郡諸軍事、建威將軍、江州刺史，從東道出鄱陽。循遣將英紏千餘人斷五畝嶠，悅破之，進據豫章，絕循糧援。
------------------------------
鎮軍府版諮議參軍，轉車騎從事中郎。劉毅請為撫軍司馬，不就。遷車騎中軍司馬。從征廣固，竭其誠力。盧循逼京都，以為督江州豫州之西陽新蔡汝南潁川司州之恒農揚州之松滋六郡諸軍事、建威將軍、江州刺史，從東道出鄱陽。循遣將英紏千餘人斷五畝嶠，<person name='庾悅'>悅</person>破之，進據豫章，絕循糧援。
初，毅家在京口，貧約過常，嘗與鄉曲士大夫往東堂共射。時悅為司徒右長史，暫至京，要府州僚佐共出東堂。毅已先至，遣與悅相聞，曰：「身久躓頓，營一遊集甚難。君如意人，無處不可為適，豈能以此堂見讓。」悅素豪，徑前，不答毅語。眾人並避之，唯毅留射如故。悅厨饌甚盛，不以及毅。毅既不去，悅甚不歡，俄頃亦退。毅又相聞曰：「身今年未得子鵝，豈能以殘炙見惠。」悅又不答。盧循平後，毅求都督江州，以江州內地

In [201]:
re.sub(r'褚叔度', f"<person name='褚叔度'>褚叔度</person>", paragraph.text)

"<person name='褚叔度'>褚叔度</person>，河南陽翟人也。曾祖裒，晉太傅。祖歆，祕書監。父爽，金紫光祿大夫。"

In [124]:
s.split_name('靈符')

(None, None)

In [172]:
'謝安' in persons

True

In [193]:
persons_list

['褚叔度', '褚秀之']